# Appendix B: Making Python Faster

## Using arrays

#### Data types

In [1]:
N = [79, 171, 265, 355]

In [2]:
N[1], type(N[1]), type(N)

(171, int, list)

In [3]:
import numpy as np

# convert to array
N = np.array(N)

print(N[1], N[1].dtype, N.dtype)

171 int64 int64


In [4]:
# redefine list 
N = [79, "summer solstice", 265, "winter solstice"]

N[1], type(N[1]), type(N)

('summer solstice', str, list)

In [5]:
N[0], type(N[0]), type(N)

(79, int, list)

#### Example: Planck spectrum

Compute a Planck spectrum using lists

In [6]:
import math
from scipy.constants import h,c,k,sigma

In [7]:
# list of wavenumbers
n = 1000
lambda_max = 2e-6
lambda_step = lambda_max/n
wavelength = [i*lambda_step for i in range(1,n+1)]

In [8]:
def planck_spectrum(wavelength, T=5778):
    
    # create empty list
    spectrum = []

    # loop over wavelengths and append flux values
    for val in wavelength:
        spectrum.append(2*h*c**2 / 
            (val**5 * (math.exp(min(700, h*c/(val*k*T))) - 1)))
        
    return spectrum

In [9]:
%timeit planck_spectrum(wavelength)

773 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Compute a Planck spectrum using arrays

In [10]:
import numpy as np

# array of wavenumbers
n = 1000
lambda_max = 2e-6
wavelength = np.linspace(lambda_max/n, lambda_max, n)

In [11]:
def planck_spectrum(wavelength, T=5778):
    return 2*h*c**2 / (wavelength**5 * 
        (np.exp(np.minimum(700, h*c/(wavelength*k*T))) - 1))

In [12]:
%timeit planck_spectrum(wavelength)

90.8 µs ± 4.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
solar = planck_spectrum(wavelength)

In [14]:
solar.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

#### Function calls

Conventional implementation of trapezoidal rule with explcit loop and single-valued function calls:

In [15]:
def integr_trapez(f, a, b, n):
    
    # integration step
    h = (b - a)/n
    
    # initialisation
    tmp = 0.5*f(a)
    
    # loop over subintervals between a+h and b-h
    for i in range(1,n):
        tmp += f(a + i*h)
        
    tmp += 0.5*f(b)
    
    return h*tmp

In [16]:
# verification
integr_trapez(math.sin, 0, math.pi/2, 20)

0.9994859052485329

In [17]:
%timeit integr_trapez(planck_spectrum, 1e-9, 364.7e-9, 100)

399 µs ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Implementation using Numpy arrays

In [18]:
import numpy as np

def integr_trapez(f, a, b, n):

    # integration step
    h = (b - a)/n
    
    # endpoints of subintervals between a+h and b-h
    x = np.linspace(a+h, b-h, n-1)
    
    return 0.5*h*(f(a) + 2*np.sum(f(x)) + f(b))

In [19]:
%timeit integr_trapez(planck_spectrum, 1e-9, 364.7e-9, 100)

67.8 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Cythonizing code

In [20]:
def rk4_step(f, t, x, dt):

    k1 = dt * f(t, x)
    k2 = dt * f(t + 0.5*dt, x + 0.5*k1)
    k3 = dt * f(t + 0.5*dt, x + 0.5*k2)
    k4 = dt * f(t + dt, x + k3) 

    return x + (k1 + 2*(k2 + k3) + k4)/6

In [21]:
import numpy as np

def solve_stroemgren(r0, dt, n_steps):
    t = np.linspace(0, n_steps*dt, n_steps+1)
    r = np.zeros(n_steps+1)
    r[0] = r0

    for n in range(n_steps):
        r[n+1] = rk4_step(lambda t, r: (1 - r**3)/(3*r**2),
                          t[n], r[n], dt)

    return (t,r)

In [22]:
%timeit solve_stroemgren(0.01, 1e-3, 10000)

64.8 ms ± 5.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


You need to run

```python setup.py build_ext --inplace```

on the command line to create the C-extension module stroemgren

In [30]:
%%bash

python setup.py build_ext --inplace

Traceback (most recent call last):
  File "setup.py", line 2, in <module>
    from Cython.Build import cythonize
ModuleNotFoundError: No module named 'Cython'


CalledProcessError: Command 'b'\npython setup.py build_ext --inplace\n'' returned non-zero exit status 1.

In [26]:
import numpy as np
from stroemgren import crk4_step

ModuleNotFoundError: No module named 'stroemgren'

In [ ]:
def solve_stroemgren(r0, dt, n_steps):
    t = np.linspace(0, n_steps*dt, n_steps+1)
    r = np.zeros(n_steps+1)
    r[0] = r0

    for n in range(n_steps):
        r[n+1] = crk4_step(lambda t, r: (1 - r**3)/(3*r**2),
                           t[n], r[n], dt)

    return (t,r)

In [ ]:
%timeit solve_stroemgren(0.01, 1e-3, 10000)

In [ ]:
from stroemgren import stroemgren_step

In [ ]:
def solve_stroemgren(r0, dt, n_steps):
    t = np.linspace(0, n_steps*dt, n_steps+1)
    r = np.zeros(n_steps+1)
    r[0] = r0

    for n in range(n_steps):
        r[n+1] = stroemgren_step(t[n], r[n], dt)

    return (t,r)

In [ ]:
%timeit solve_stroemgren(0.01, 1e-3, 10000)